# **¿Tiene un mejor rendimiento un portafolio generado aleatoriamente que un fondo gestionado activamente?**

* ### Base de datos de ETF sacados de: https://etfdb.com/themes/actively-managed-etfs/

In [1]:
import pandas as pd
import yfinance as yf
import random
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from pypfopt.expected_returns import mean_historical_return
from statsmodels.stats.power import TTestIndPower

sns.set_palette("dark")
warnings.filterwarnings("ignore")


id	year	returns	random_portfolio	volatility	sector
1	2020	0.12	0	0.18	Tecnología
2	2020	0.08	1	0.10	Energía

*********
## **Obteniendo y limpiando datos**

In [2]:
etf_data = pd.read_csv("C:\\Users\\kike\\OneDrive - Universidad de Alcala\\Documentos\\Entorno Jupyter\\Proyectos y prácticas\\Articulo\\etfs_details_type_fund_flow.csv")
random_portfolios = pd.read_csv("C:\\Users\\kike\\OneDrive - Universidad de Alcala\\Documentos\\Entorno Jupyter\\Proyectos y prácticas\\Articulo\\portafolios_aleatorios.csv")

etf_data.columns
print(etf_data.isnull().sum())
etfs = etf_data[etf_data["Asset Class"] == "Equity"]
etfs_list = etfs["Symbol"].tolist()

Symbol                                           0
ETF Name                                         0
Asset Class                                      5
Total Assets                                     0
YTD Price Change                                43
Avg. Daily Volume                               42
Previous Closing Price                          27
1-Day Change                                     0
Inverse                                          0
Leveraged                                        0
1 Week                                          31
1 Month                                        156
1 Year                                          78
3 Year                                         438
5 Year                                         851
YTD FF                                           0
1 Week FF                                        0
4 Week FF                                        0
1 Year FF                                        0
3 Year FF                      

In [11]:
# A period of 5 years
start = dt.datetime(2020, 1, 1)
end = dt.datetime(2025, 1, 1)   

etfs_validos = []
etfs_invalidos = []

for i, etf in enumerate(etfs_list):
    try:
        data = yf.download(etf, start=start, end=end, progress=False, interval = "1mo")
        
        print(f"Iteration {i + 1}/{len(etfs_list)}: {etf}")

        if not data.empty:
            if len(data) >=  58:  
                etfs_validos.append(etf)
                print(f"✅ {etf}: Valid (complete 5-year data)")
            else:
                etfs_invalidos.append(etf)
                print(f"⚠️ {etf}: Insufficient data ({len(data)} trading months)")
        else:
            etfs_invalidos.append(etf)
            print(f"❌ {etf}: Empty dataset (likely delisted)")
            
    except Exception as e:
        etfs_invalidos.append(etf)
        print(f"❌ {etf}: Download error ({str(e)})")

Iteration 1/717: JEPI
⚠️ JEPI: Insufficient data (56 trading months)
Iteration 2/717: DFAC
⚠️ DFAC: Insufficient data (43 trading months)
Iteration 3/717: JEPQ
⚠️ JEPQ: Insufficient data (32 trading months)
Iteration 4/717: DYNF
✅ DYNF: Valid (complete 5-year data)
Iteration 5/717: CGDV
⚠️ CGDV: Insufficient data (35 trading months)
Iteration 6/717: AVUV
✅ AVUV: Valid (complete 5-year data)
Iteration 7/717: DFUS
⚠️ DFUS: Insufficient data (43 trading months)
Iteration 8/717: DFUV
⚠️ DFUV: Insufficient data (32 trading months)
Iteration 9/717: CGGR
⚠️ CGGR: Insufficient data (35 trading months)
Iteration 10/717: DFIV
⚠️ DFIV: Insufficient data (40 trading months)
Iteration 11/717: DFAT
⚠️ DFAT: Insufficient data (43 trading months)
Iteration 12/717: DFAI
⚠️ DFAI: Insufficient data (50 trading months)
Iteration 13/717: DFIC
⚠️ DFIC: Insufficient data (34 trading months)
Iteration 14/717: AVDV
✅ AVDV: Valid (complete 5-year data)
Iteration 15/717: AVEM
✅ AVEM: Valid (complete 5-year data)


1 Failed download:
['TTFS']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-01-01 00:00:00 -> 2025-01-01 00:00:00)')


Iteration 299/717: TTFS
❌ TTFS: Empty dataset (likely delisted)
Iteration 300/717: STNC
⚠️ STNC: Insufficient data (46 trading months)
Iteration 301/717: EMC
⚠️ EMC: Insufficient data (20 trading months)
Iteration 302/717: AVDS
⚠️ AVDS: Insufficient data (18 trading months)
Iteration 303/717: YDEC
⚠️ YDEC: Insufficient data (49 trading months)
Iteration 304/717: AAPU
⚠️ AAPU: Insufficient data (29 trading months)
Iteration 305/717: CRPT
⚠️ CRPT: Insufficient data (40 trading months)
Iteration 306/717: PSMR
⚠️ PSMR: Insufficient data (45 trading months)
Iteration 307/717: FRTY
⚠️ FRTY: Insufficient data (46 trading months)
Iteration 308/717: TMFS
✅ TMFS: Valid (complete 5-year data)
Iteration 309/717: DWAW
✅ DWAW: Valid (complete 5-year data)
Iteration 310/717: GLRY
⚠️ GLRY: Insufficient data (49 trading months)
Iteration 311/717: EJUL
✅ EJUL: Valid (complete 5-year data)
Iteration 312/717: EOCT
⚠️ EOCT: Insufficient data (39 trading months)
Iteration 313/717: PSMJ
⚠️ PSMJ: Insufficient


1 Failed download:
['HECO']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-01-01 00:00:00 -> 2025-01-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1577854800, endDate = 1735707600")')


Iteration 365/717: HECO
❌ HECO: Empty dataset (likely delisted)
Iteration 366/717: MKOR
⚠️ MKOR: Insufficient data (18 trading months)
Iteration 367/717: AOTG
⚠️ AOTG: Insufficient data (31 trading months)
Iteration 368/717: SAGP
⚠️ SAGP: Insufficient data (36 trading months)
Iteration 369/717: UMAY
⚠️ UMAY: Insufficient data (56 trading months)
Iteration 370/717: OAEM
⚠️ OAEM: Insufficient data (28 trading months)
Iteration 371/717: APRT
⚠️ APRT: Insufficient data (55 trading months)
Iteration 372/717: SURE
✅ SURE: Valid (complete 5-year data)
Iteration 373/717: SPRX
⚠️ SPRX: Insufficient data (41 trading months)
Iteration 374/717: LCF
⚠️ LCF: Insufficient data (29 trading months)
Iteration 375/717: JANZ
⚠️ JANZ: Insufficient data (48 trading months)
Iteration 376/717: HIDV
⚠️ HIDV: Insufficient data (22 trading months)
Iteration 377/717: DIVS
⚠️ DIVS: Insufficient data (45 trading months)
Iteration 378/717: PSCQ
⚠️ PSCQ: Insufficient data (39 trading months)
Iteration 379/717: SSXU
⚠

...

Iteration 710/717: IELG
❌ IELG: Empty dataset (likely delisted)
Iteration 711/717: IESM
❌ IESM: Empty dataset (likely delisted)
Iteration 712/717: HDGI
❌ HDGI: Empty dataset (likely delisted)


['IEIL']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-01-01 00:00:00 -> 2025-01-01 00:00:00)')

1 Failed download:
['IEIS']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-01-01 00:00:00 -> 2025-01-01 00:00:00)')

1 Failed download:
['EPRO']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-01-01 00:00:00 -> 2025-01-01 00:00:00)')

1 Failed download:


Iteration 713/717: IEIL
❌ IEIL: Empty dataset (likely delisted)
Iteration 714/717: IEIS
❌ IEIS: Empty dataset (likely delisted)
Iteration 715/717: EPRO
❌ EPRO: Empty dataset (likely delisted)


['CFGE']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2020-01-01 00:00:00 -> 2025-01-01 00:00:00)')


Iteration 716/717: CFGE
❌ CFGE: Empty dataset (likely delisted)
Iteration 717/717: EVSTCNX
⚠️ EVSTCNX: Insufficient data (35 trading months)


In [13]:
print(f"\tValidation Completed\n{"-" * 35}")
print(f"Total ETFs checked: {len(etfs_list)}")
print(f"Valid ETFs (5 years data available): {len(etfs_validos)}")
print(f"Invalid/insufficient data ETFs: {len(etfs_invalidos)}")


	Validation Completed
-----------------------------------
Total ETFs checked: 717
Valid ETFs (5 years data available): 148
Invalid/insufficient data ETFs: 569


In [ ]:
# Downloading the data for the valid ETFs
prices_etfs = yf.download(etfs_validos,
                        start = start,
                        end = end,
                        progress = False,
                        interval = "1mo")["Close"]
prices_etfs.head()

Ticker,AADR,ABEQ,ACGR,AESR,AFLG,AFMC,AFSM,AIVI,AIVL,AMOM,...,VICE,VMOT,VWID,WBIF,WBIG,WBIL,WTV,XLSR,YLDE,YOLO
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,50.213768,22.954782,29.079664,9.241175,19.093687,18.891605,18.923414,32.403210,77.692375,20.905783,...,23.215454,22.693941,20.226147,25.189587,21.442524,25.667631,41.526104,30.566288,30.059269,10.364358
2020-02-01,46.062717,21.361910,26.695770,8.403649,17.257225,17.095570,17.083252,29.498854,69.713844,19.381214,...,21.365431,20.552767,18.596458,24.133083,20.529144,24.736160,37.084862,27.986237,27.364954,8.651830
2020-03-01,37.630013,18.929058,25.383183,7.615548,14.846473,13.634351,13.200169,24.687037,56.318836,18.123680,...,18.037279,19.193165,15.837837,23.670044,20.140162,24.255955,29.515591,24.911055,24.162186,6.894707
2020-04-01,42.900452,20.567816,27.697397,8.495181,16.751469,15.629148,15.536168,26.503235,63.401062,20.434305,...,20.717447,19.910330,17.514473,23.771154,20.213032,24.459547,34.099369,27.875788,27.119827,7.845012
2020-05-01,45.792198,21.013556,29.036177,8.895182,17.604864,16.126879,15.838860,27.412779,64.631744,22.445410,...,21.837896,20.625357,18.492134,24.612194,20.781466,25.436230,35.946671,28.992260,27.841309,9.749583


In [ ]:
prices_etfs.to_csv("prices_etf.csv", index = True)

In [ ]:
print("Are there null or missing values?")
for etf in prices_etfs.columns:
	print(f"{etf}: {prices_etfs[etf].isnull().sum()}")

Are there null or missing values?
AADR: 0
ABEQ: 0
ACGR: 0
AESR: 0
AFLG: 0
AFMC: 0
AFSM: 0
AIVI: 0
AIVL: 0
AMOM: 0
AMZA: 0
ARKF: 0
ARKG: 0
ARKK: 0
ARKQ: 0
ARKW: 0
AVDE: 0
AVDV: 0
AVEM: 0
AVUS: 0
AVUV: 0
BAPR: 0
BATT: 0
BAUG: 0
BDEC: 0
BFEB: 1
BJAN: 0
BJUL: 0
BJUN: 0
BLOK: 0
BNOV: 0
BOCT: 0
BSEP: 0
BUYZ: 1
CCOR: 0
CNBS: 0
CWS: 0
DAUG: 0
DFNL: 0
DGRE: 0
DINT: 0
DIVI: 0
DIVO: 0
DSTL: 0
DUSA: 0
DWAW: 0
DWLD: 0
DWMF: 0
DWSH: 0
DWUS: 0
DYNF: 0
ECLN: 0
EJAN: 0
EJUL: 0
EMLP: 0
EMMF: 0
EQIN: 0
ERUS: 0
EYLD: 0
FAUG: 0
FM: 0
FTHI: 0
FYLD: 0
GVAL: 0
HDGE: 0
HDMV: 0
HELX: 1
HUSV: 0
IEDI: 0
IETC: 0
IJAN: 0
IJUL: 0
IMOM: 0
INEQ: 0
IQM: 1
IVAL: 0
KJAN: 0
KOCT: 0
LRGE: 0
MOTO: 0
NJAN: 0
NOCT: 0
NUSI: 0
OSCV: 0
OVF: 0
OVL: 0
OVS: 0
PAPR: 0
PAUG: 0
PDEC: 0
PFEB: 1
PHDG: 0
PJAN: 0
PJUL: 0
PJUN: 0
PNOV: 0
POCT: 0
PSC: 0
PSEP: 0
QMOM: 0
QQQH: 0
QRFT: 0
QVAL: 0
RFDA: 0
RFDI: 0
RFEM: 0
RFEU: 0
RFFC: 0
SECT: 0
SMCP: 0
SSUS: 0
SURE: 0
SYLD: 0
TMFS: 0
TOKE: 0
TTAC: 0
TTAI: 0
UAPR: 0
UAUG: 0
UDEC: 0
UFEB: 1
UJAN: 